# 1、ChatMessageHistory的使用

场景1：记忆存储


In [1]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import PromptTemplate

# 1、ChatMessageHistory的实例化
history = InMemoryChatMessageHistory()

# 2、添加相关的消息进行存储

history.add_user_message("你好")

history.add_ai_message("很高兴认识你")

# 3、打印存储的消息
print(history.messages)

[HumanMessage(content='你好', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]


In [2]:
# 获取大模型
import os
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")

# 创建大模型实例
llm = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

In [3]:
from langchain_core.chat_history import InMemoryChatMessageHistory

history = InMemoryChatMessageHistory()

# 2、添加相关的消息进行存储
history.add_user_message("你好")
history.add_ai_message("很高兴认识你")
history.add_user_message("帮我计算1 + 2 * 3 = ？")

response = llm.invoke(history.messages)
print(response.content)

根据数学中的运算规则，乘法优先于加法。所以，1 + 2 * 3 应该先计算乘法部分：

2 * 3 = 6

然后将结果加到1上：

1 + 6 = 7

因此，1 + 2 * 3 的结果是 7。


# 2、ConversationBufferMemory的使用（老方法被淘汰了）

举例1：以字符串的方式返回存储的信息

In [7]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage

# 1、ConversationBufferMemory的实例化
memory = InMemoryChatMessageHistory()

# 2、存储相关的消息
memory.add_message(HumanMessage(content="你好，我叫小明"))
memory.add_message(AIMessage(content="很高兴认识你"))

memory.add_message(HumanMessage(content="帮我回答一下1+2*3=?"))
memory.add_message(AIMessage(content="7"))

# 3、获取存储的信息
print(memory.messages)

# 说明：返回的字典结构的key叫history

[HumanMessage(content='你好，我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[]), HumanMessage(content='帮我回答一下1+2*3=?', additional_kwargs={}, response_metadata={}), AIMessage(content='7', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]


举例2：以消息列表的方式返回存储的信息

In [9]:
from langchain_core.chat_history import InMemoryChatMessageHistory

store = {} 

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 1. 获取历史对象 (传入任意 session_id)
history = get_session_history("test_user_1")

# 2. 添加消息
history.add_user_message("这是用户说的话：1+1等于几？")
history.add_ai_message("这是AI说的话：等于2")

# 3. 打印结果
print(history.messages)

[HumanMessage(content='这是用户说的话：1+1等于几？', additional_kwargs={}, response_metadata={}), AIMessage(content='这是AI说的话：等于2', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]


举例3：结合大模型、提示词模板的使用（PromptTemplate）

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, AIMessage
import os
import dotenv
dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("LLM_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("LLM_BASE_URL")

# 1. 创建模型
llm = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

# 2. Prompt (使用 MessagesPlaceholder 替代手动拼接 history)
prompt = ChatPromptTemplate.from_messages([
    ("system", "你可以与人类对话。"),
    MessagesPlaceholder(variable_name="history"), # 这是一个占位符，专门放历史记录
    ("human", "{question}"),
])

# 3. 定义 Chain (使用 LCEL 管道符 | 替代 LLMChain)
# 逻辑流：Prompt -> 模型 -> 输出转字符串
chain = prompt | llm | StrOutputParser()

# 4. 记忆管理 (替代 ConversationBufferMemory)
# 用一个简单的字典模拟数据库
user_1_history = InMemoryChatMessageHistory()
user_1_history.add_message(HumanMessage(content="小明喜欢什么？"))
user_1_history.add_message(AIMessage(content="小明喜欢编程。"))
store = {"user_1": user_1_history}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 使用 RunnableWithMessageHistory 包装 chain
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
)

# 5. 调用
response = chain_with_history.invoke(
    {"question": "你好，我的名字叫小明，你知道我喜欢什么吗？"},
    config={"configurable": {"session_id": "user_1"}}
)

print(response)

你好，小明！根据我们之前的对话，你提到你喜欢编程。不过，如果你还有其他特别的兴趣爱好或者喜欢的事物，也随时可以告诉我哦！


举例4：基于举例3，显式的设置meory的key的值


In [20]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.output_parsers import StrOutputParser

# 1、创建大模型实例
llm = ChatOpenAI(model=os.getenv("LLM_MODEL_ID"))

# 2、提供提示词模板 (ChatPromptTemplate 替代 PromptTemplate)
prompt = ChatPromptTemplate.from_messages([
    ("system", "你可以与人类对话。"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}"),
])

# 3、提供Chain (使用管道符 | 构建)
chain = prompt | llm | StrOutputParser()

# 4、提供memory管理逻辑 (替代 ConversationBufferMemory)
store = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 5、包装 Chain
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",      # 对应输入的 key
    history_messages_key="chat_history",  # 对应提示词模板中的占位符
)

# 6、调用
response_1 = chain_with_history.invoke(
    {"question": "你好，我的名字叫小罗,我喜欢小鹿。"},
    config={"configurable": {"session_id": "user_1"}}
)

response_2 = chain_with_history.invoke(
    {"question": "你好，我的名字叫小谢，你知道我喜欢什么吗，小罗喜欢的我不喜欢？"},
    config={"configurable": {"session_id": "user_2"}}
)

print(store)
print(store["user_1"])
print(type(store["user_1"]))
print(response_2)

{'user_1': InMemoryChatMessageHistory(messages=[HumanMessage(content='你好，我的名字叫小罗,我喜欢小鹿。', additional_kwargs={}, response_metadata={}), AIMessage(content='你好小罗，很高兴认识你。你对小鹿有这样的喜好，可能是因为它们温顺、可爱，还是因为其他的原因呢？', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])]), 'user_2': InMemoryChatMessageHistory(messages=[HumanMessage(content='你好，我的名字叫小谢，你知道我喜欢什么吗，小罗喜欢的我不喜欢？', additional_kwargs={}, response_metadata={}), AIMessage(content='你好小谢！通过这句话，你是否是在说你和小罗在兴趣爱好上有差异呢？不过，我不是你们的专属想象力助手，无法直接得知你的具体喜好是什么。但是，一般人都会有自己独特的喜好，如音乐、电影、书籍、运动等等，而这些可能会和你身边的人有所不同。例如你可能非常喜欢科幻类的电影，而小罗则可能更喜欢爱情电影。如果你愿意的话，可以具体说一下你的喜好，我可以帮助你找到和小罗共享的共同兴趣。', additional_kwargs={}, response_metadata={}, tool_calls=[], invalid_tool_calls=[])])}
Human: 你好，我的名字叫小罗,我喜欢小鹿。
AI: 你好小罗，很高兴认识你。你对小鹿有这样的喜好，可能是因为它们温顺、可爱，还是因为其他的原因呢？
<class 'langchain_core.chat_history.InMemoryChatMessageHistory'>
你好小谢！通过这句话，你是否是在说你和小罗在兴趣爱好上有差异呢？不过，我不是你们的专属想象力助手，无法直接得知你的具体喜好是什么。但是，一般人都会有自己独特的喜好，如音乐、电影、书籍、运动等等，而这些可能会和你身边的人有所不同。例如你可能非常喜欢科幻类的电影，而